<a href="https://colab.research.google.com/github/quangtrung0220/SieuAmTim/blob/master/SieuAmTim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### ***Import các thư viện cần thiết***

In [69]:
import torch
from torch import nn
from torch.utils.data import DataLoader

from torchvision import transforms
import torchvision

import matplotlib.pyplot as plt

from collections import namedtuple

from sklearn.metrics import classification_report

import cv2

import numpy as np


### ***Clone git để lấy dữ liệu ảnh train và test***

In [70]:
# Mount với drive để lấy datasets 
# from google.colab import drive
# drive.mount('/content/drive')

# Link git lấy dataset 
!git clone https://github.com/quangtrung0220/SieuAmTim.git
traindir = "/content/SieuAmTim/DATA_CHAMBER_2021/train"
testdir = "/content/SieuAmTim/DATA_CHAMBER_2021/test"


fatal: destination path 'SieuAmTim' already exists and is not an empty directory.


### ***Lấy ra các nhãn - classes và chuẩn bị dữ liệu***


---



*   Các nhãn: 2C, 3C, 4C
*   Các loại kích cỡ ảnh train và test: 224x224, 32x32
*   Preprocess: Cân bằng sáng: histogram, Lọc nhiễu: GaussianBlur
*   Data Augmentation: RandomCrop, RandomHorizontalFlip, RandomVerticalFlip
*   Đưa vào batch để tận dụng khả năng xử lý song song của GPU
*   Folder train và test lần lượt trong: traindir và testdir






In [71]:
# Hàm lấy ra các classes
def get_clases():
  classes = ['2C', '3C', '4C']
  return classes

TrainTest = namedtuple('TrainTest', ['train', 'test'])

# Hàm chuẩn bị dữ liệu
def prepare_data():

  # Preprocess
  gaussianBlur = transforms.GaussianBlur(3)
  histogramEqualize = transforms.RandomEqualize(p=0.5)

  # #Normalize
  # norm = transforms.Normalize(mean=[0.485, 0.456, 0.406],
  #      std=[0.229, 0.224, 0.225])

  #Augmentation for all
  horizontal = transforms.RandomHorizontalFlip()
  vertical = transforms.RandomVerticalFlip()

  #Image 224x224
  resize_224 = transforms.Resize((224,224))
  crop_224 = transforms.RandomCrop(224, padding=4)
  
  #Image 32x32
  resize_32 = transforms.Resize((32,32))
  crop_32 = transforms.RandomCrop(32, padding=4)

  #To tensor
  tensor = transforms.ToTensor()

  #Raw image
  transform_train_raw224 = transforms.Compose([resize_224, tensor])
  transform_train_raw232 = transforms.Compose([resize_32, tensor])

  #Augmentation image
  transform_train_aug224 = transforms.Compose([resize_224, crop_224, horizontal, vertical, tensor])
  transform_train_aug32 = transforms.Compose([resize_32, crop_32, horizontal, vertical, tensor])

  #Preprocess image
  transform_train_pre224 = transforms.Compose([resize_224, gaussianBlur, histogramEqualize, tensor])
  transform_train_pre32 = transforms.Compose([resize_32, gaussianBlur, histogramEqualize, tensor])

  #Test image
  transform_test_224 = transforms.Compose([resize_224, tensor])
  transform_test_32 = transforms.Compose([resize_32, tensor])

  # transform_test = transforms.Compose([
  #   transforms.Resize((224,224)),
  #   # transforms.Resize((32,32)),

  #   # Preprocess
  #   transforms.GaussianBlur(3),
  #   transforms.RandomEqualize(p=0.5),

  #   transforms.ToTensor(),
  #   transforms.Normalize(mean=[0.485, 0.456, 0.406],
  #      std=[0.229, 0.224, 0.225])
  # ])
  trainset = torchvision.datasets.ImageFolder(root=traindir, transform=transform_train_aug224)
  testset = torchvision.datasets.ImageFolder(root=testdir, transform=transform_test_224)
  return TrainTest(train=trainset, test=testset)

# Hàm load dữ liệu
def prepare_loader(datasets):
  trainloader = DataLoader(dataset=datasets.train, batch_size=35, shuffle=True, num_workers=4)
  testloader = DataLoader(dataset=datasets.test, batch_size=35, shuffle=False, num_workers=4)
  return TrainTest(train=trainloader, test=testloader)

### ***Hàm train và test***


---



*   BatchSize = 35
*   Report sau mỗi 20 epoch 
*   Train: cho đi qua model và tính lỗi bằng loss_func sau đó cập nhật tham số mới
*   Test: khai báo chế độ eval(đánh giá - evaluate) và trả về nhãn dự đoán và nhãn thực tế



In [ ]:
# Data = prepare_data()
# img, label = Data.train[3000]
# print(label)
# plt.show(img)
# print(img)
# s = Data.train.imgs[0]
# print(s)

In [ ]:
# Hàm train 
def train_epoch(epoch, model, loader, loss_func, optimizer, device):
  true_result = []
  pred_result = []

  # gọi hàm train để model biết mình phải cần huấn luyện 
  model.train()
  running_loss = 0.0
  reporting_steps = 20
  for i, (images, labels) in enumerate(loader):
    images, labels = images.to(device), labels.to(device)
    outputs = model(images)
    true_result += list(labels.cpu().numpy())
    _, predicted = torch.max(outputs, dim=1)
    pred_result += list(predicted.cpu().numpy())
    # Tính lỗi 
    loss = loss_func(outputs, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    if i % reporting_steps == reporting_steps-1:
      print(f"Epoch {epoch} step {i} ave_loss {running_loss/reporting_steps:.4f}")
      running_loss = 0.0
  # return pred_result, true_result

# hàm test 
def test_epoch(epoch, model, loader, device):
  ytrue = []
  ypred = []
  # Gọi hàm eval để model biết đang ở chế độ test 
  with torch.no_grad():
    model.eval()
    
    for i, (images, labels) in enumerate(loader):
      images, labels = images.to(device), labels.to(device)
      outputs = model(images)
      _, predicted = torch.max(outputs, dim=1)

      ytrue += list(labels.cpu().numpy())
      ypred += list(predicted.cpu().numpy())

  return ypred, ytrue

### ***Hàm main***

---


*   model_in = 'vgg16', 'resnet50' hoặc 'desnet': tên mạng Cnn đùng để huấn luyện
*   PATH: đường dẫn lưu trọng số sau khi huấn luyện
*   Đặc trưng ra = 3 vì có 3 lớp 2C, 3C, 4C
*   Hàm lỗi được sử dụng: Cross-Entropy
*   Hàm tối ưu đước sử dụng: SGD - Stochastic Gradient Descent




In [ ]:
def main(PATH='./model.pth', model_in=''):
  classes = get_clases()
  datasets = prepare_data()
  # img, label = datasets.train[0]
  # plt.imshow(img)
  # print(classes[label], img.size)
  # print('train', len(datasets.train), 'test', len(datasets.test))
  
  loaders = prepare_loader(datasets)

  # images, labels = iter(loaders.train).next()
  # print(images.shape, labels.shape)

  device = torch.device("cuda:0")
  torch.cuda.empty_cache()

  # model = ResNet50().to(device)
  # model = VGG16().to(device)
  # images, labels = iter(loaders.train).next()
  # outputs = model(images)
  # print(outputs.shape)
  # print(outputs[0])
  # _, predicted = torch.max(outputs, dim=1)
  # print(predicted)
  # imshow(images, labels, predicted, classes)

# Load model 
  if model_in == 'vgg16':
    model = torchvision.models.vgg16()
    model.classifier[6] = torch.nn.modules.linear.Linear(in_features=4096, out_features=3, bias=True)
    # if torch.cuda.is_available():
    model.to(device=device)
  elif model_in == 'resnet50':
    model = torchvision.models.resnet50(pretrained=False, progress=False)
    model.fc = torch.nn.modules.linear.Linear(in_features=2048, out_features=3, bias=True)  
    # if torch.cuda.is_available():
    model.to(device=device)
  elif model_in == 'desnet':
    model = torchvision.models.densenet121(pretrained=False, progress=False)
    model.classifier = torch.nn.modules.linear.Linear(in_features=1024, out_features=3, bias=True)
    # if torch.cuda.is_available():
    model.to(device=device)
  else:
    pass  

  # array_accuracy = []

  loss_func = nn.CrossEntropyLoss()
  optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
  for epoch in range(10):
    print(f'Epoch {epoch} report: ')
    
    # pred_result, true_result = 
    train_epoch(epoch, model, loaders.train, loss_func, optimizer, device)
    # print('Train report: ')
    # print(classification_report(true_result, pred_result, target_names=classes))

    print('Test report: ')
    ypred, ytrue = test_epoch(epoch, model, loaders.test, device)

    # accuracy = (ytrue_test==ypred_test).sum() / len(ytrue_test)
    # array_accuracy.append(accuracy)

    print(classification_report(ytrue, ypred, target_names=classes))

    torch.save(model.state_dict(), PATH)

  return model

model = main(PATH="./desnet121.pth", model_in='desnet')

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch 0 report: 
Epoch 0 step 19 ave_loss 1.0896
Epoch 0 step 39 ave_loss 0.9752
Epoch 0 step 59 ave_loss 0.9268
Epoch 0 step 79 ave_loss 0.8415
Epoch 0 step 99 ave_loss 0.8537
Epoch 0 step 119 ave_loss 0.9380
Epoch 0 step 139 ave_loss 0.7161
Epoch 0 step 159 ave_loss 0.5320
Epoch 0 step 179 ave_loss 0.4864
Test report: 
              precision    recall  f1-score   support

          2C       0.45      0.82      0.58       409
          3C       0.23      0.51      0.32       367
          4C       1.00      0.05      0.10       831

    accuracy                           0.35      1607
   macro avg       0.56      0.46      0.33      1607
weighted avg       0.68      0.35      0.27      1607

Epoch 1 report: 


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch 1 step 19 ave_loss 0.4579
Epoch 1 step 39 ave_loss 0.4649
Epoch 1 step 59 ave_loss 0.4700
Epoch 1 step 79 ave_loss 0.3703
Epoch 1 step 99 ave_loss 0.2924
Epoch 1 step 119 ave_loss 0.2788
Epoch 1 step 139 ave_loss 0.2997
Epoch 1 step 159 ave_loss 0.2077
Epoch 1 step 179 ave_loss 0.2292
Test report: 


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

Epoch 7 step 19 ave_loss 0.0171
Epoch 7 step 39 ave_loss 0.0054
Epoch 7 step 59 ave_loss 0.0057
Epoch 7 step 79 ave_loss 0.0208
Epoch 7 step 99 ave_loss 0.0143
Epoch 7 step 119 ave_loss 0.0157
Epoch 7 step 139 ave_loss 0.0180
Epoch 7 step 159 ave_loss 0.0257
Epoch 7 step 179 ave_loss 0.0165
Test report: 
              precision    recall  f1-score   support

          2C       0.84      0.64      0.73       409
          3C       0.56      0.87      0.68       367
          4C       0.94      0.81      0.87       831

    accuracy                           0.78      1607
   macro avg       0.78      0.78      0.76      1607
weighted avg       0.83      0.78      0.79      1607

Epoch 8 report: 


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch 8 step 19 ave_loss 0.0233
Epoch 8 step 39 ave_loss 0.0193
Epoch 8 step 59 ave_loss 0.0138
Epoch 8 step 79 ave_loss 0.0170
Epoch 8 step 99 ave_loss 0.0174
Epoch 8 step 119 ave_loss 0.0214
Epoch 8 step 139 ave_loss 0.0111
Epoch 8 step 159 ave_loss 0.0200
Epoch 8 step 179 ave_loss 0.0117
Test report: 
              precision    recall  f1-score   support

          2C       0.56      0.88      0.69       409
          3C       0.80      0.90      0.84       367
          4C       1.00      0.67      0.80       831

    accuracy                           0.77      1607
   macro avg       0.79      0.81      0.78      1607
weighted avg       0.84      0.77      0.78      1607

Epoch 9 report: 


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch 9 step 19 ave_loss 0.0091
Epoch 9 step 39 ave_loss 0.0134
Epoch 9 step 59 ave_loss 0.0054
Epoch 9 step 79 ave_loss 0.0087
Epoch 9 step 99 ave_loss 0.0100
Epoch 9 step 119 ave_loss 0.0221
Epoch 9 step 139 ave_loss 0.0184
Epoch 9 step 159 ave_loss 0.0242
Epoch 9 step 179 ave_loss 0.0101
Test report: 
              precision    recall  f1-score   support

          2C       0.57      1.00      0.72       409
          3C       0.93      0.75      0.83       367
          4C       0.94      0.67      0.78       831

    accuracy                           0.77      1607
   macro avg       0.81      0.81      0.78      1607
weighted avg       0.84      0.77      0.78      1607



### ***Test Code ***

In [89]:
# Data = prepare_data()
# frame_id_list = []
# for i, image in enumerate(Data.test.imgs):
#     id = image[0].split("/")[-1].split("_")[0]
#     frame_id_list.append(id)
# print(frame_id_list) 

['158', '158', '158', '158', '158', '158', '158', '158', '158', '158', '158', '158', '158', '158', '158', '158', '158', '158', '158', '158', '158', '158', '158', '158', '158', '158', '158', '158', '158', '158', '158', '158', '158', '158', '158', '158', '165', '165', '165', '165', '165', '165', '165', '165', '165', '165', '165', '165', '165', '165', '165', '165', '165', '165', '165', '165', '165', '165', '165', '165', '165', '165', '165', '165', '165', '165', '165', '168', '168', '168', '168', '168', '168', '168', '168', '168', '168', '168', '168', '168', '168', '168', '168', '168', '168', '168', '168', '169', '169', '169', '169', '169', '169', '169', '169', '169', '169', '169', '169', '169', '169', '169', '169', '169', '169', '169', '169', '169', '169', '169', '169', '169', '169', '169', '169', '169', '169', '169', '169', '169', '169', '169', '169', '171', '171', '171', '171', '171', '171', '171', '171', '171', '171', '171', '171', '171', '171', '171', '171', '171', '171', '171', '171'

In [ ]:
# video_list = [] 
# for frame in frame_id_list:
#   if (frame in video_list) == False:
#     video_list.append(frame)
#   else:
#        pass
# print(video_list)

['158', '165', '168', '169', '171', '176', '177', '178', '181', '183', '191', '192', '157', '159', '161', '162', '166', '174', '175', '179', '185', '186', '189', '190', '194', '160', '163', '164', '167', '170', '172', '173', '180', '182', '184', '187', '188', '193', '195']


In [ ]:
a = [1,2,3,2,3,1,1,1,1,1,3,2]
values_arr, counts_arr = np.unique(a, return_counts=True)
print(values_arr)
print(counts_arr)
result = np.where(counts_arr == np.max(counts_arr))
print(values_arr[result])

[1 2 3]
[6 3 3]
[1]


### ***Phân lớp với Video***


---



*   Với kết quả huấn luyện bên trên thì cho ảnh qua ta sẽ có kết quả dự đoán lớp của ảnh input mới
*   1 video sẽ có nhiều frame nên kết quả phân lớp video sẽ là số nhãn xuất hiện nhiều nhất trong số các frame của video
*   Giả sử video A có 90% nhãn được dự đoán là lớp 2C => video thuộc lớp 2C2C




In [103]:
def video_classify(model=None, testdir=None, device='cuda'):

  # check testsets
  if testdir == None:
    testdir = torchvision.datasets.ImageFolder(root=testdir, 
                                               transform=ransforms.Compose([resize_224, transforms.ToTensor()]))

  video = namedtuple('video', ['id', "label_true", 'label_pred'])

  # get id list
  frame_id_list = []
  for i, image in enumerate(testdir.imgs):
    id = image[0].split("/")[-1].split("_")[0]
    frame_id_list.append(id)

  # get list video
  video_list = np.unique(frame_id_list, return_counts=False)

  ytrue = []
  ypred = []
  model.to(device)
  with torch.no_grad():
    model.eval()
    
    for images, labels in testdir:
      images = images.unsqueeze(0).to(device)
      outputs = model(images)
      _, predicted = torch.max(outputs, dim=1)
      ytrue.append(labels)
      ypred += list(predicted.cpu().numpy())

  # if id in frame_id_list = id in video_list: add yred and ytrue to pred_vid and true_vid
  outputs = []
  ytrue_video = []
  ypred_video = []
  for id in video_list:
    true_vid = []
    pred_vid = []
    for index, img in enumerate(frame_id_list):
      if img == id:
        pred_vid.append(ypred[index])
        true_vid.append(ytrue[index])

    # classification
    value_true, count_true = np.unique(true_vid, return_counts=True)
    label_true = value_true[np.where(count_true == np.max(count_true))]

    value_pred, count_pred = np.unique(pred_vid, return_counts=True)
    label_pred = value_pred[np.where(count_pred == np.max(count_pred))]

    print("id:", id, ", true:", label_true, ", pred:",label_pred)
    ytrue_video.append(label_true)
    ypred_video.append(label_pred)
    outputs.append(video(id=id, label_true=label_true, label_pred=label_pred))

  classes = get_clases()
  print(classification_report(ytrue_video, ypred_video, target_names=classes))
  return outputs


In [104]:
Data = prepare_data()
testset = Data.test
video_classify(model=model, testdir=testset, device='cuda')

id: 157 , true: [1] , pred: [1]
id: 158 , true: [0] , pred: [0]
id: 159 , true: [1] , pred: [1]
id: 160 , true: [2] , pred: [2]
id: 161 , true: [1] , pred: [1]
id: 162 , true: [1] , pred: [1]
id: 163 , true: [2] , pred: [2]
id: 164 , true: [2] , pred: [2]
id: 165 , true: [0] , pred: [0]
id: 166 , true: [1] , pred: [1]
id: 167 , true: [2] , pred: [2]
id: 168 , true: [0] , pred: [0]
id: 169 , true: [0] , pred: [0]
id: 170 , true: [2] , pred: [0]
id: 171 , true: [0] , pred: [0]
id: 172 , true: [2] , pred: [2]
id: 173 , true: [2] , pred: [2]
id: 174 , true: [1] , pred: [0]
id: 175 , true: [1] , pred: [1]
id: 176 , true: [0] , pred: [0]
id: 177 , true: [0] , pred: [0]
id: 178 , true: [0] , pred: [0]
id: 179 , true: [1] , pred: [0]
id: 180 , true: [2] , pred: [2]
id: 181 , true: [0] , pred: [0]
id: 182 , true: [2] , pred: [2]
id: 183 , true: [0] , pred: [0]
id: 184 , true: [2] , pred: [2]
id: 185 , true: [1] , pred: [1]
id: 186 , true: [1] , pred: [1]
id: 187 , true: [2] , pred: [0]
id: 188 

[video(id='157', label_true=array([1]), label_pred=array([1])),
 video(id='158', label_true=array([0]), label_pred=array([0])),
 video(id='159', label_true=array([1]), label_pred=array([1])),
 video(id='160', label_true=array([2]), label_pred=array([2])),
 video(id='161', label_true=array([1]), label_pred=array([1])),
 video(id='162', label_true=array([1]), label_pred=array([1])),
 video(id='163', label_true=array([2]), label_pred=array([2])),
 video(id='164', label_true=array([2]), label_pred=array([2])),
 video(id='165', label_true=array([0]), label_pred=array([0])),
 video(id='166', label_true=array([1]), label_pred=array([1])),
 video(id='167', label_true=array([2]), label_pred=array([2])),
 video(id='168', label_true=array([0]), label_pred=array([0])),
 video(id='169', label_true=array([0]), label_pred=array([0])),
 video(id='170', label_true=array([2]), label_pred=array([0])),
 video(id='171', label_true=array([0]), label_pred=array([0])),
 video(id='172', label_true=array([2]), 